In [1]:
import gutenbergpy.textget
import pandas as pd 

In [2]:
def download_text(id):
    try:
        text = gutenbergpy.textget.get_text_by_id(id)
        text = gutenbergpy.textget.strip_headers(text)
        text = text.decode('utf-8')
        return text
    except Exception as e:
        print("error downloading ID", id)
        raise e

In [3]:
df_metadata = pd.read_csv('gutenberg.csv')
#filter type text
df_metadata = df_metadata[df_metadata['Type'] == 'Text']
#filter langauge english
df_metadata = df_metadata[df_metadata['Language'] == 'en']
#get only Text Authors and Title
df_metadata = df_metadata[['Text#','Authors','Title']]
df_metadata = df_metadata.dropna().reset_index(drop=True)
#remove years
df_metadata["Authors"] = df_metadata["Authors"].str.split(',').str[:-1].str.join(',').str.strip()
df_metadata["Authors"]


/tmp/ipykernel_546738/3277692249.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_metadata = pd.read_csv('gutenberg.csv')


0                         Jefferson, Thomas
1                                          
2        Kennedy, John F. (John Fitzgerald)
3                          Lincoln, Abraham
4                                          
                        ...                
55596                   Schiller, Friedrich
55597                        Huneker, James
55598              Flandrau, Charles Macomb
55599         Cornell, Frederick Carruthers
55600                                      
Name: Authors, Length: 55601, dtype: object

Seleccionamos los autores que están en el top 100
https://www.gutenberg.org/browse/scores/top

In [4]:
authors = pd.read_csv('authors.csv')
#find last '(' and remove everything after
authors['Author'] = authors['Author'].str.split('(').str[:-1].str.join('(').str.strip()

authors


,Author
0,"Shakespeare, William"
1,"Smollett, T. (Tobias)"
2,"Forster, E. M. (Edward Morgan)"
3,"Montgomery, L. M. (Lucy Maud)"
4,"Alcott, Louisa May"
...,...
95,"Balagtas, Francisco"
96,"Marcus Aurelius, Emperor of Rome"
97,"Wodehouse, P. G. (Pelham Grenville)"
98,"Wharton, Edith"


In [5]:
_df_metadata_filtered = df_metadata[df_metadata['Authors'].isin(authors['Author'])]



In [6]:
#pick a random book per author with seed 13
df_metadata_filtered = _df_metadata_filtered.groupby('Authors').sample(n=1, random_state=13).reset_index(drop=True)


In [7]:
len(df_metadata_filtered)

72

In [8]:
code_ids = df_metadata_filtered['Text#'].tolist()
books = [download_text(id) for id in code_ids]

In [9]:
#add books column to dataframe
df_metadata_filtered['Books'] = books

In [10]:
df_metadata_filtered

,Text#,Authors,Title,Books
0,37807,"Alcott, Louisa May",Mountain-Laurel and Maidenhair,"\n\n\n\n\n\n\n\n\n [Illustration: ""Her eyes b..."
1,1212,"Austen, Jane",Love and Freindship [sic],\n\n\n\nLOVE & FREINDSHIP\nAND\nOTHER EARLY WO...
2,6861,"Balzac, Honoré de",Vautrin: A Drama in Five Acts,\n\n\n\n\n VAUTR...
3,39543,"Barrie, J. M. (James Matthew)","A Holiday in Bed, and Other Sketches",\n\n\n\n\n\nA HOLIDAY IN BED\n\nAnd other Sket...
4,961,"Baum, L. Frank (Lyman Frank)",Glinda of Oz\r\nIn Which Are Related the Excit...,\n\n\n\n\n\n\n\n\nGLINDA OF OZ\n\n\nby\n\nL. F...
...,...,...,...,...
67,30855,"Wells, H. G. (Herbert George)",The Wife of Sir Isaac Harman,\n\n\n\nTHE WIFE OF\nSIR ISAAC HARMAN\n\nBY\nH...
68,166,"Wharton, Edith",Summer,\n\n\n\n\nSUMMER\n\nby Edith Wharton\n\n\n1917...
69,4001,"Widger, David",Widger's Quotations from Project Gutenberg Edi...,\n\n\n\n\nWIDGER'S QUOTATIONS\n\nFROM THE PROJ...
70,902,"Wilde, Oscar","The Happy Prince, and Other Tales",\n [Picture: Book cov...


In [11]:
#save dataframe to csv
df_metadata_filtered.to_csv('gutenberg_with_text.csv', index=False)